In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np

In [3]:
buoy_id = 45176
url = 'http://greatlakesbuoys.org/select_range.php?station=' + str(buoy_id)
result = requests.get(url)
if result.status_code != 200:
    print("error on status code")
soup = BeautifulSoup(result.content, "html5lib")

In [7]:
a = soup.find('form', attrs={'id':'dateForm'})
b = a

<form ;="" action="genData.php" id="dateForm" method="post" onsubmit="return validateForm(this)"> 


<b>Instructions:</b><br/>
To download buoy data, please select the desired start date and end date of the data to be downloaded.  <br/>Next, click on "Get Data" to initiate a download of a
comma-seperated value (CSV) file including all buoy data collected between the selected dates.<br/><br/>
<br/>
Download the full list of field descriptions <a href="./FieldDescriptions.txt"> here</a>.<br/><br/>



-First date available: 06/07/2016 12:40:00<br/>Latest date available: 05/13/2017 18:40:00<br/><br/>
<table>
<tbody><tr><td><b>Select start date:</b></td></tr>
<tr><td>  

Month:
<select id="startMonth" name="startMonth">
<option value="00">--
</option><option value="01">Jan
</option><option value="02">Feb
</option><option value="03">Mar
</option><option value="04">Apr
</option><option value="05">May
</option><option value="06">Jun
</option><option value="07">Jul
</option><option value="08">A

In [2]:
# The name on the page, the name we want to use
columns_of_interest = {'ID': 'id', 'DATE': 'timestamp', 'YSIBGARFU': 'blue_green_algae',
                      'YSITEMP': 'water_temperature'}

In [117]:
def update_df_to_floats(data, exclusion_list):
    for column in data.columns:
        if column not in exclusion_list:
            data[column] = data[column].astype(float)
    return data

In [177]:
def build_df_from_data_ta(raw_rows):
    data = []
    
    headings = [_.text.split('(')[0] for _ in raw_rows[1].findAll('th')]
    significant_columns = {}
    for i, heading in enumerate(headings):
        if heading in [_ for _ in columns_of_interest]:
            significant_columns[i] = columns_of_interest[heading]
    print(significant_columns)
    for row_raw in raw_rows[2:]:

        row = [_.text for _ in row_raw.findAll('td')]
        data.append({significant_columns[column]: row[column] for column in significant_columns.keys()})
    data = pd.DataFrame(data)
    data = data.replace(to_replace=r'-', value=np.nan).replace(to_replace=r'---', value=np.nan)
    # data = update_df_to_floats(data, ['id', 'mean_wave_direction', 'wind_direction', 'timestamp'])
    # data['timestamp'] = pd.to_datetime(data['timestamp'])
    return data

In [178]:
def scrape_buoy_data(buoy_id):
    url = 'http://greatlakesbuoys.org/station_page.php?station=%s&unit=E&tz=CST' % str(buoy_id)
    result = requests.get(url)
    if result.status_code != 200:
        print("error on status code")
    soup = BeautifulSoup(result.content, "html5lib")
    raw_rows = soup.find(id="prevNObs").find('table').findAll('tr')
    data = build_df_from_data_ta(raw_rows)
    data['buoy_id'] = buoy_id
    return data

In [181]:
result = scrape_buoy_data(45176)
result.head()

{0: 'id', 1: 'timestamp', 22: 'water_temperature', 28: 'blue_green_algae'}


,blue_green_algae,id,timestamp,water_temperature,buoy_id
0,0.32,27324,05/08/2017 20:30:00,53.05,45176
1,0.32,27323,05/08/2017 20:20:00,53.05,45176
2,0.32,27322,05/08/2017 20:10:00,53.05,45176
3,0.32,27321,05/08/2017 20:00:00,53.05,45176
4,0.32,27320,05/08/2017 19:50:00,53.05,45176
